In [20]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp
from matplotlib import pyplot as plt
import seaborn as sns
import mglearn
%matplotlib inline
from IPython.display import display
from pprint import pprint
pd.set_option('display.max_columns', 50)
DATA_PATH='D:\download'
t_e = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions_Movregress.csv'),
    
)
t_e['DEV0']=t_e['DEV0'].astype(np.int16)
t_e['DEV1']=t_e['DEV1'].astype(np.int16)
t_e['DEV2']=t_e['DEV2'].astype(np.int16)
t_e['DEV3']=t_e['DEV3'].astype(np.int16)
t_e['DEV4']=t_e['DEV4'].astype(np.int16)
t_e['DEV5']=t_e['DEV5'].astype(np.int16)
t_e['DEV6']=t_e['DEV6'].astype(np.int16)
t_e['feature_5_65']=t_e['feature_5_65'].astype(np.int16)
t_e['feature_5_00']=t_e['feature_5_00'].astype(np.int16)
t_e['feature_5_68']=t_e['feature_5_68'].astype(np.int16)
t_e['feature_5_59']=t_e['feature_5_59'].astype(np.int16)
t_e['feature_5_-1']=t_e['feature_5_-1'].astype(np.int16)
t_e['smallf3']= t_e['smallf3'].astype(np.int16)
t_e['middlef3']=t_e['middlef3'].astype(np.int16)
t_e['bigf3']=t_e['bigf3'].astype(np.int16)
t_e['CM_S']=t_e['CM_S'].astype(np.int16)
t_e['CM_P']=t_e['CM_P'].astype(np.int16)
t_e['CM_R']=t_e['CM_R'].astype(np.int16)
t_e['feature_5_44']=t_e['feature_5_44'].astype(np.int16)
t_e['feature_3']=t_e['feature_3'].astype(np.int16)
t_e['duration']=t_e['duration'].astype(np.int16)
t_e['element_uid']=t_e['element_uid'].astype(np.int32)
t_e['user_uid']=t_e['user_uid'].astype(np.int32)
t_e=t_e.drop(['Unnamed: 0'],1)
t_e.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8447928 entries, 0 to 8447927
Data columns (total 33 columns):
element_uid       int32
user_uid          int32
watched_time_x    float64
rating            float64
duration          int16
feature_2         float64
feature_3         int16
feature_4         float64
quantity          int64
rating_mean       float64
CM_S              int16
CM_P              int16
CM_R              int16
feature_5_65      int16
feature_5_00      int16
feature_5_68      int16
feature_5_59      int16
feature_5_-1      int16
feature_5_44      int16
smallf3           int16
middlef3          int16
bigf3             int16
longfilm          int64
shortfilm         int64
ts-f1             float64
watched_time_y    int64
DEV0              int16
DEV1              int16
DEV2              int16
DEV3              int16
DEV4              int16
DEV5              int16
DEV6              int16
dtypes: float64(6), int16(21), int32(2), int64(4)
memory usage: 1.0 GB


In [8]:
t_e.duration= t_e.duration.map(lambda x: 5 if x == 0 else x)
t_e.watched_time_x=(t_e.watched_time_y/60)/(t_e.duration)

t_e.watched_time_y=(t_e.watched_time_y/60)
t_e['ts-f1']=t_e['ts-f1'].abs()
t_e.rating[t_e.watched_time_x > 125] = t_e.rating[t_e.watched_time_x > 125].fillna(9.0)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
from collections import Counter
t_e["user_rating_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").rating.mean())
coun = pd.DataFrame.from_dict(Counter(t_e['user_uid']), orient='index')
coun['user_uid']=coun.index
coun['quantity_user_views']=coun[0]

coun=coun.reset_index()

coun=coun.drop(['index'], axis=1)
coun=coun.drop([0],1)
t_e=pd.merge(t_e, coun, on='user_uid')

coun.head()

,user_uid,quantity_user_views
0,5177,14
1,349202,7
2,117582,8
3,523833,6
4,571064,6


In [10]:
t_e["wt_y_user_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").watched_time_y.mean())
t_e["wt_x_user_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").watched_time_x.mean())
t_e["new_content"]=t_e['ts-f1'].map(lambda x: 1 if x<31 else 0)

In [11]:
t_e=t_e.drop(['longfilm','shortfilm'],1)

t_e['quartile__element_views'] = pd.qcut(t_e['quantity'], 4, ['1','2','3','4'])
t_e['quartile__user_views'] = pd.qcut(t_e['quantity_user_views'], 4, ['1','2','3','4'])
t_e['quartile__duration'] = pd.qcut(t_e['duration'], 4, ['1','2','3','4'])
t_e['quartile__wtx'] = pd.qcut(t_e['wt_x_user_mean'], 4, ['1','2','3','4'])
t_e["Q_el_views=1"]=t_e['quartile__element_views'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_el_views=4"]=t_e['quartile__element_views'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_us_views=1"]=t_e['quartile__user_views'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_us_views=4"]=t_e['quartile__user_views'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_duration=1"]=t_e['quartile__duration'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_duration=4"]=t_e['quartile__duration'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_wtx=1"]=t_e['quartile__wtx'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_wtx=4"]=t_e['quartile__wtx'].map(lambda x: 1 if x=='4' else 0)

t_e['quartile__f2'] = pd.qcut(t_e['feature_2'], 4, ['1','2','3','4'])
t_e['quartile__f4'] = pd.qcut(t_e['feature_4'], 4, ['1','2','3','4'])

t_e["Q_f2=1"]=t_e['quartile__f2'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_f2=4"]=t_e['quartile__f2'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_f4=1"]=t_e['quartile__f4'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_f4=4"]=t_e['quartile__f4'].map(lambda x: 1 if x=='4' else 0)
t_e=t_e.drop(['quartile__element_views', 'quartile__user_views',  'quartile__duration', 'quartile__wtx', 'quartile__f2', 'quartile__f4'],1)

In [12]:
t_e['new_content']=t_e['new_content'].astype(np.int16)
t_e['Q_el_views=1']=t_e['Q_el_views=1'].astype(np.int16)
t_e['Q_el_views=4']=t_e['Q_el_views=4'].astype(np.int16)
t_e['Q_us_views=1']=t_e['Q_us_views=1'].astype(np.int16)
t_e['Q_us_views=4']=t_e['Q_us_views=4'].astype(np.int16)
t_e['Q_duration=1']=t_e['Q_duration=1'].astype(np.int16)
t_e['Q_duration=4']=t_e['Q_duration=4'].astype(np.int16)
t_e['Q_wtx=1']=t_e['Q_wtx=1'].astype(np.int16)
t_e['Q_wtx=4']=t_e['Q_wtx=4'].astype(np.int16)
t_e['Q_f2=1']=t_e['Q_f2=1'].astype(np.int16)
t_e['Q_f2=4']=t_e['Q_f2=1'].astype(np.int16)
t_e['Q_f4=1']=t_e['Q_f4=1'].astype(np.int16)
t_e['Q_f4=4']=t_e['Q_f4=1'].astype(np.int16)
t_e.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8447928 entries, 0 to 8447927
Data columns (total 48 columns):
element_uid            int32
user_uid               int32
watched_time_x         float64
rating                 float64
duration               int64
feature_2              float64
feature_3              int16
feature_4              float64
quantity               int64
rating_mean            float64
CM_S                   int16
CM_P                   int16
CM_R                   int16
feature_5_65           int16
feature_5_00           int16
feature_5_68           int16
feature_5_59           int16
feature_5_-1           int16
feature_5_44           int16
smallf3                int16
middlef3               int16
bigf3                  int16
ts-f1                  float64
watched_time_y         float64
DEV0                   int16
DEV1                   int16
DEV2                   int16
DEV3                   int16
DEV4                   int16
DEV5                   int16
DEV

In [7]:
t_e.isnull().sum()/t_e.shape[0]

element_uid            0.000000
user_uid               0.000000
watched_time_x         0.000000
rating                 0.958629
duration               0.000000
feature_2              0.000000
feature_3              0.000000
feature_4              0.000000
quantity               0.000000
rating_mean            0.002783
CM_S                   0.000000
CM_P                   0.000000
CM_R                   0.000000
feature_5_65           0.000000
feature_5_00           0.000000
feature_5_68           0.000000
feature_5_59           0.000000
feature_5_-1           0.000000
feature_5_44           0.000000
smallf3                0.000000
middlef3               0.000000
bigf3                  0.000000
ts-f1                  0.000000
watched_time_y         0.000000
DEV0                   0.000000
DEV1                   0.000000
DEV2                   0.000000
DEV3                   0.000000
DEV4                   0.000000
DEV5                   0.000000
DEV6                   0.000000
user_rat

In [8]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

t_e.user_rating_mean=t_e.user_rating_mean.fillna(6.0) 
transactions_exp=t_e

transactions_exp.rating_mean=transactions_exp.rating_mean.fillna(6.0) 

transactions_exp['worst']=transactions_exp.watched_time_x.map(lambda x:1 if (x < 0.2 and x >0.005) else 0)
transactions_exp[transactions_exp.worst == 1]=transactions_exp[transactions_exp.worst == 1].fillna(2.0)
transactions_exp=transactions_exp.drop(['worst'],1)
transactions_exp['watched']=transactions_exp.watched_time_x.map(lambda x: 0 if x < 0.51 else 1)


X_train = transactions_exp[~transactions_exp.rating.isnull()]
X_train=X_train.drop(['rating'],1)
X_test = transactions_exp[transactions_exp.rating.isnull()]
X_test=X_test.drop(['rating'],1)
y_train = transactions_exp[~transactions_exp.rating.isnull()].rating.astype(np.float)
y_test = transactions_exp[transactions_exp.rating.isnull()].rating




#kf = KFold(n_splits=3, shuffle=True, random_state=241) 


model2 = RandomForestRegressor( n_estimators=522, max_depth=8, random_state=241, n_jobs=-1)
#score=cross_val_score(model2, X_train, y_train, cv=kf, n_jobs=-1, scoring='mean_squared_error')
#print("Score is", score.mean())



#score1=cross_val_score(estimator, X_train, y_train, cv=kf, n_jobs=-1, scoring='mean_squared_error')
#print("Score is", score1.mean())




D:\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [ ]:
- 3.32 примерно было без кучи признаков

In [ ]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor



transactions_exp=t_e
transactions_exp2=t_e
transactions_exp3=t_e
transactions_exp4=t_e
transactions_exp5=t_e


print('strat')

transactions_exp.rating_mean=transactions_exp.rating_mean.fillna(5.0) 
transactions_exp.user_rating_mean=transactions_exp.user_rating_mean.fillna(5.0) 
transactions_exp['worst']=transactions_exp.watched_time_x.map(lambda x:1 if (x < 0.2 and x >0.005) else 0)
transactions_exp[transactions_exp.worst == 1]=transactions_exp[transactions_exp.worst == 1].fillna(2.0)
transactions_exp=transactions_exp.drop(['worst'],1)
transactions_exp['watched']=transactions_exp.watched_time_x.map(lambda x: 0 if x < 0.51 else 1)


X_train = transactions_exp[~transactions_exp.rating.isnull()]
X_train=X_train.drop(['rating'],1)
X_test = transactions_exp[transactions_exp.rating.isnull()]
X_test=X_test.drop(['rating'],1)
y_train = transactions_exp[~transactions_exp.rating.isnull()].rating.astype(np.float)
y_test = transactions_exp[transactions_exp.rating.isnull()].rating





model2 = RandomForestRegressor( n_estimators=522, max_depth=8, random_state=241, n_jobs=-1)
model2.fit(X_train, y_train)
submit1 = pd.DataFrame(model2.predict(X_test))

estimator=xgb.XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=700, n_jobs=-1  )    
estimator.fit(X_train, y_train)
submit2 = pd.DataFrame(estimator.predict(X_test))



X_test=X_test.drop([
'watched_time_x',
'watched',
'duration',
'feature_2',
'feature_3',
'user_rating_mean',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV2',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_duration=1',
'Q_duration=4',
'Q_wtx=1',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4',
'Q_us_views=1',
'Q_us_views=4'],1)
X_test=X_test.reset_index()
X_test=X_test.drop(['index'], 1)

OLD=pd.concat([X_test, submit1], 1)
OLD['rating']=OLD[0]
OLD=OLD.drop([0],1)
#ratings=ratings.drop(['ts'],1)
X_train = transactions_exp[~transactions_exp.rating.isnull()]
X_train=X_train.drop(['watched',
'watched_time_x',
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV2',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_us_views=1',
'Q_us_views=4',
'Q_duration=1',
'Q_duration=4',
'Q_wtx=1',
'user_rating_mean',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4'],1)
X_train=X_train.reset_index()
X_train=X_train.drop(['index'], 1)
OLD=OLD.append(X_train)
OLD.to_csv("MovRatingRF1.csv")

OLD2=pd.concat([X_test, submit2], 1)
OLD2['rating']=OLD2[0]
OLD2=OLD2.drop([0],1)
OLD2=OLD2.append(X_train)
OLD2.to_csv("MovRatingXGB1.csv")

print('1')

transactions_exp2.rating_mean=transactions_exp2.rating_mean.fillna(4.0) 
transactions_exp2.user_rating_mean=transactions_exp2.user_rating_mean.fillna(4.0) 
transactions_exp2['worst']=transactions_exp2.watched_time_x.map(lambda x:1 if (x < 0.2 and x >0.005) else 0)
transactions_exp2[transactions_exp2.worst == 1]=transactions_exp2[transactions_exp2.worst == 1].fillna(2.0)
transactions_exp2=transactions_exp2.drop(['worst'],1)
transactions_exp2['watched']=transactions_exp2.watched_time_x.map(lambda x: 0 if x < 0.51 else 1)


X_train = transactions_exp2[~transactions_exp2.rating.isnull()]
X_train=X_train.drop(['rating'],1)
X_test = transactions_exp2[transactions_exp2.rating.isnull()]
X_test=X_test.drop(['rating'],1)
y_train = transactions_exp2[~transactions_exp2.rating.isnull()].rating.astype(np.float)
y_test = transactions_exp2[transactions_exp2.rating.isnull()].rating





model2 = RandomForestRegressor( n_estimators=522, max_depth=8, random_state=241, n_jobs=-1)
model2.fit(X_train, y_train)
submit1 = pd.DataFrame(model2.predict(X_test))

estimator=xgb.XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=700, n_jobs=-1  )    
estimator.fit(X_train, y_train)
submit2 = pd.DataFrame(estimator.predict(X_test))



X_test=X_test.drop([
'watched_time_x',
'watched',
'duration',
'feature_2',
'feature_3',
'user_rating_mean',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV2',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_duration=1',
'Q_duration=4',
'Q_wtx=1',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4',
'Q_us_views=1',
'Q_us_views=4'],1)
X_test=X_test.reset_index()
X_test=X_test.drop(['index'], 1)

OLD=pd.concat([X_test, submit1], 1)
OLD['rating']=OLD[0]
OLD=OLD.drop([0],1)
#ratings=ratings.drop(['ts'],1)
X_train = transactions_exp2[~transactions_exp2.rating.isnull()]
X_train=X_train.drop(['watched',
'watched_time_x',
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV2',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_us_views=1',
'Q_us_views=4',
'Q_duration=1',
'Q_duration=4',
'Q_wtx=1',
'user_rating_mean',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4'],1)
X_train=X_train.reset_index()
X_train=X_train.drop(['index'], 1)
OLD=OLD.append(X_train)
OLD.to_csv("MovRatingRF2.csv")

OLD2=pd.concat([X_test, submit2], 1)
OLD2['rating']=OLD2[0]
OLD2=OLD2.drop([0],1)
OLD2=OLD2.append(X_train)
OLD2.to_csv("MovRatingXGB2.csv")


print('2')
transactions_exp3.rating_mean=transactions_exp3.rating_mean.fillna(7.0) 
transactions_exp3.user_rating_mean=transactions_exp3.user_rating_mean.fillna(7.0) 
transactions_exp3['worst']=transactions_exp3.watched_time_x.map(lambda x:1 if (x < 0.2 and x >0.005) else 0)
transactions_exp3[transactions_exp3.worst == 1]=transactions_exp3[transactions_exp3.worst == 1].fillna(2.0)
transactions_exp3=transactions_exp3.drop(['worst'],1)
transactions_exp3['watched']=transactions_exp3.watched_time_x.map(lambda x: 0 if x < 0.51 else 1)


X_train = transactions_exp3[~transactions_exp3.rating.isnull()]
X_train=X_train.drop(['rating'],1)
X_test = transactions_exp3[transactions_exp3.rating.isnull()]
X_test=X_test.drop(['rating'],1)
y_train = transactions_exp3[~transactions_exp3.rating.isnull()].rating.astype(np.float)
y_test = transactions_exp3[transactions_exp3.rating.isnull()].rating





model2 = RandomForestRegressor( n_estimators=522, max_depth=8, random_state=241, n_jobs=-1)
model2.fit(X_train, y_train)
submit1 = pd.DataFrame(model2.predict(X_test))

estimator=xgb.XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=700, n_jobs=-1  )    
estimator.fit(X_train, y_train)
submit2 = pd.DataFrame(estimator.predict(X_test))



X_test=X_test.drop([
'watched_time_x',
'watched',
'duration',
'feature_2',
'feature_3',
'user_rating_mean',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV2',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_duration=1',
'Q_duration=4',
'Q_wtx=1',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4',
'Q_us_views=1',
'Q_us_views=4'],1)
X_test=X_test.reset_index()
X_test=X_test.drop(['index'], 1)

OLD=pd.concat([X_test, submit1], 1)
OLD['rating']=OLD[0]
OLD=OLD.drop([0],1)
#ratings=ratings.drop(['ts'],1)
X_train = transactions_exp3[~transactions_exp3.rating.isnull()]
X_train=X_train.drop(['watched',
'watched_time_x',
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV2',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_us_views=1',
'Q_us_views=4',
'Q_duration=1',
'Q_duration=4',
'Q_wtx=1',
'user_rating_mean',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4'],1)
X_train=X_train.reset_index()
X_train=X_train.drop(['index'], 1)
OLD=OLD.append(X_train)
OLD.to_csv("MovRatingRF3.csv")

OLD2=pd.concat([X_test, submit2], 1)
OLD2['rating']=OLD2[0]
OLD2=OLD2.drop([0],1)
OLD2=OLD2.append(X_train)
OLD2.to_csv("MovRatingXGB3.csv")

print('3')

transactions_exp4.rating_mean=transactions_exp4.rating_mean.fillna(6.0) 
transactions_exp4=transactions_exp4.drop(['user_rating_mean'],1)
transactions_exp4['worst']=transactions_exp4.watched_time_x.map(lambda x:1 if (x < 0.2 and x >0.005) else 0)
transactions_exp4[transactions_exp4.worst == 1]=transactions_exp4[transactions_exp4.worst == 1].fillna(2.0)
transactions_exp4=transactions_exp4.drop(['worst'],1)
transactions_exp4['watched']=transactions_exp4.watched_time_x.map(lambda x: 0 if x < 0.51 else 1)


X_train = transactions_exp4[~transactions_exp4.rating.isnull()]
X_train=X_train.drop(['rating'],1)
X_test = transactions_exp4[transactions_exp4.rating.isnull()]
X_test=X_test.drop(['rating'],1)
y_train = transactions_exp4[~transactions_exp4.rating.isnull()].rating.astype(np.float)
y_test = transactions_exp4[transactions_exp4.rating.isnull()].rating





model2 = RandomForestRegressor( n_estimators=522, max_depth=8, random_state=241, n_jobs=-1)
model2.fit(X_train, y_train)
submit1 = pd.DataFrame(model2.predict(X_test))

estimator=xgb.XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=700, n_jobs=-1  )    
estimator.fit(X_train, y_train)
submit2 = pd.DataFrame(estimator.predict(X_test))



X_test=X_test.drop([
'watched_time_x',
'watched',
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV2',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_duration=1',
'Q_duration=4',
'Q_wtx=1',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4',
'Q_us_views=1',
'Q_us_views=4'],1)
X_test=X_test.reset_index()
X_test=X_test.drop(['index'], 1)

OLD=pd.concat([X_test, submit1], 1)
OLD['rating']=OLD[0]
OLD=OLD.drop([0],1)
#ratings=ratings.drop(['ts'],1)
X_train = transactions_exp4[~transactions_exp4.rating.isnull()]
X_train=X_train.drop(['watched',
'watched_time_x',
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV2',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_us_views=1',
'Q_us_views=4',
'Q_duration=1',
'Q_duration=4',
'Q_wtx=1',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4'],1)
X_train=X_train.reset_index()
X_train=X_train.drop(['index'], 1)
OLD=OLD.append(X_train)
OLD.to_csv("MovRatingRF4.csv")

OLD2=pd.concat([X_test, submit2], 1)
OLD2['rating']=OLD2[0]
OLD2=OLD2.drop([0],1)
OLD2=OLD2.append(X_train)
OLD2.to_csv("MovRatingXGB4.csv")

print('4')


transactions_exp5.rating_mean=transactions_exp5.rating_mean.fillna(6.0) 
transactions_exp5.user_rating_mean=transactions_exp5.user_rating_mean.fillna(6.0) 
transactions_exp5['worst']=transactions_exp5.watched_time_x.map(lambda x:1 if (x < 0.2 and x >0.005) else 0)
transactions_exp5[transactions_exp5.worst == 1]=transactions_exp5[transactions_exp5.worst == 1].fillna(2.0)
                                         

transactions_exp5['worst1']=transactions_exp5.watched_time_x.map(lambda x:1 if (x < 0.5 and x >0.199999) else 0)
transactions_exp5[transactions_exp5.worst1 == 1]=transactions_exp5[transactions_exp5.worst1 == 1].fillna(4.0)                                         
transactions_exp5=transactions_exp5.drop(['worst', 'worst1'],1)
transactions_exp5['watched']=transactions_exp5.watched_time_x.map(lambda x: 0 if x < 0.51 else 1)


X_train = transactions_exp5[~transactions_exp5.rating.isnull()]
X_train=X_train.drop(['rating'],1)
X_test = transactions_exp5[transactions_exp5.rating.isnull()]
X_test=X_test.drop(['rating'],1)
y_train = transactions_exp5[~transactions_exp5.rating.isnull()].rating.astype(np.float)
y_test = transactions_exp5[transactions_exp5.rating.isnull()].rating





model2 = RandomForestRegressor( n_estimators=522, max_depth=8, random_state=241, n_jobs=-1)
model2.fit(X_train, y_train)
submit1 = pd.DataFrame(model2.predict(X_test))

estimator=xgb.XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=700, n_jobs=-1  )    
estimator.fit(X_train, y_train)
submit2 = pd.DataFrame(estimator.predict(X_test))



X_test=X_test.drop([
'watched_time_x',
'watched',
'duration',
'feature_2',
'feature_3',
'user_rating_mean',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV2',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_duration=1',
'Q_duration=4',
'Q_wtx=1',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4',
'Q_us_views=1',
print('end')


D:\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


strat
1
2
3
4


In [13]:
transactions_exp=t_e
transactions_exp2=t_e
transactions_exp3=t_e
transactions_exp4=t_e
transactions_exp5=t_e


print('strat')

transactions_exp.rating_mean=transactions_exp.rating_mean.fillna(5.0) 
transactions_exp.user_rating_mean=transactions_exp.user_rating_mean.fillna(5.0) 
transactions_exp['worst']=transactions_exp.watched_time_x.map(lambda x:1 if (x < 0.2 and x >0.005) else 0)
transactions_exp[transactions_exp.worst == 1]=transactions_exp[transactions_exp.worst == 1].fillna(2.0)
transactions_exp=transactions_exp.drop(['worst'],1)
transactions_exp['watched']=transactions_exp.watched_time_x.map(lambda x: 0 if x < 0.51 else 1)



transactions_exp2.rating_mean=transactions_exp2.rating_mean.fillna(4.0) 
transactions_exp2.user_rating_mean=transactions_exp2.user_rating_mean.fillna(4.0) 
transactions_exp2['worst']=transactions_exp2.watched_time_x.map(lambda x:1 if (x < 0.2 and x >0.005) else 0)
transactions_exp2[transactions_exp2.worst == 1]=transactions_exp2[transactions_exp2.worst == 1].fillna(2.0)
transactions_exp2=transactions_exp2.drop(['worst'],1)
transactions_exp2['watched']=transactions_exp2.watched_time_x.map(lambda x: 0 if x < 0.51 else 1)

transactions_exp3.rating_mean=transactions_exp3.rating_mean.fillna(7.0) 
transactions_exp3.user_rating_mean=transactions_exp3.user_rating_mean.fillna(7.0) 
transactions_exp3['worst']=transactions_exp3.watched_time_x.map(lambda x:1 if (x < 0.2 and x >0.005) else 0)
transactions_exp3[transactions_exp3.worst == 1]=transactions_exp3[transactions_exp3.worst == 1].fillna(2.0)
transactions_exp3=transactions_exp3.drop(['worst'],1)
transactions_exp3['watched']=transactions_exp3.watched_time_x.map(lambda x: 0 if x < 0.51 else 1)


transactions_exp4.rating_mean=transactions_exp4.rating_mean.fillna(6.0) 
transactions_exp4=transactions_exp4.drop(['user_rating_mean'],1)
transactions_exp4['worst']=transactions_exp4.watched_time_x.map(lambda x:1 if (x < 0.2 and x >0.005) else 0)
transactions_exp4[transactions_exp4.worst == 1]=transactions_exp4[transactions_exp4.worst == 1].fillna(2.0)
transactions_exp4=transactions_exp4.drop(['worst'],1)
transactions_exp4['watched']=transactions_exp4.watched_time_x.map(lambda x: 0 if x < 0.51 else 1)



transactions_exp5.rating_mean=transactions_exp5.rating_mean.fillna(6.0) 
transactions_exp5.user_rating_mean=transactions_exp5.user_rating_mean.fillna(6.0) 
transactions_exp5['worst']=transactions_exp5.watched_time_x.map(lambda x:1 if (x < 0.2 and x >0.005) else 0)
transactions_exp5[transactions_exp5.worst == 1]=transactions_exp5[transactions_exp5.worst == 1].fillna(2.0)
                                         

transactions_exp5['worst1']=transactions_exp5.watched_time_x.map(lambda x:1 if (x < 0.5 and x >0.199999) else 0)
transactions_exp5[transactions_exp5.worst1 == 1]=transactions_exp5[transactions_exp5.worst1 == 1].fillna(4.0)                                         
transactions_exp5=transactions_exp5.drop(['worst', 'worst1'],1)
transactions_exp5['watched']=transactions_exp5.watched_time_x.map(lambda x: 0 if x < 0.51 else 1)



strat


In [ ]:
3728	201681

In [27]:
transactions_exp3

,element_uid,user_uid,watched_time_x,rating,duration,feature_2,feature_3,feature_4,quantity,rating_mean,CM_S,CM_P,CM_R,feature_5_65,feature_5_00,feature_5_68,feature_5_59,feature_5_-1,feature_5_44,smallf3,middlef3,bigf3,ts-f1,watched_time_y,DEV0,DEV1,DEV2,DEV3,DEV4,DEV5,DEV6,user_rating_mean,quantity_user_views,wt_y_user_mean,wt_x_user_mean,new_content,Q_el_views=1,Q_el_views=4,Q_us_views=1,Q_us_views=4,Q_duration=1,Q_duration=4,Q_wtx=1,Q_wtx=4,Q_f2=1,Q_f2=4,Q_f4=1,Q_f4=4,watched
0,3336,5177,0.792963,9.0,90,0.739609,45,1.141929,45646,8.328548,1,0,0,1,0,0,0,0,0,0,0,1,108.010646,71.366667,1,0,0,0,0,0,0,7.666667,14,76.719048,0.797434,0,0,1,1,0,1,0,0,0,0,0,0,0,1
1,1911,5177,0.934697,NaN,110,0.731940,24,1.140273,18142,8.587455,1,0,0,0,0,1,0,0,0,0,1,0,186.866073,102.816667,1,0,0,0,0,0,0,7.666667,14,76.719048,0.797434,0,0,1,1,0,0,0,0,0,0,0,0,0,1
2,1659,5177,2.029833,NaN,100,0.721765,8,1.138604,1152,8.400000,0,1,0,1,0,0,0,0,0,1,0,0,432.654793,202.983333,1,0,0,0,0,0,0,7.666667,14,76.719048,0.797434,0,1,0,1,0,0,0,0,0,0,0,0,0,1
3,2468,5177,1.048889,9.0,90,0.729653,32,1.141929,18898,8.034254,1,0,0,1,0,0,0,0,0,0,1,0,89.850365,94.400000,1,0,0,0,0,0,0,7.666667,14,76.719048,0.797434,0,0,1,1,0,1,0,0,0,0,0,0,0,1
4,6573,5177,0.925000,4.0,60,0.492775,14,1.141929,19980,3.747440,1,0,0,0,0,1,0,0,0,1,0,0,53.715092,55.500000,1,0,0,0,0,0,0,7.666667,14,76.719048,0.797434,0,0,1,1,0,1,0,0,0,1,1,0,0,1
5,5035,5177,0.016667,2.0,90,0.729194,13,1.140273,27220,7.125926,1,0,0,0,0,1,0,0,0,1,0,0,149.181501,1.500000,1,0,0,0,0,0,0,7.666667,14,76.719048,0.797434,0,0,1,1,0,1,0,0,0,0,0,0,0,0
6,3783,5177,0.903182,NaN,110,0.766254,41,1.138604,55228,8.599783,1,0,0,1,0,0,0,0,0,0,0,1,341.549739,99.350000,1,0,0,0,0,0,0,7.666667,14,76.719048,0.797434,0,0,1,1,0,0,0,0,0,0,0,0,0,1
7,1799,5177,0.037500,2.0,80,0.739609,22,1.140273,12256,8.286432,1,0,0,1,0,0,0,0,0,0,1,0,79.436276,3.000000,1,0,0,0,0,0,0,7.666667,14,76.719048,0.797434,0,0,0,1,0,1,0,0,0,0,0,0,0,0
8,4290,5177,1.034167,NaN,80,0.642772,25,1.140273,19547,6.705051,1,0,0,1,0,0,0,0,0,0,1,0,113.574319,82.733333,1,0,0,0,0,0,0,7.666667,14,76.719048,0.797434,0,0,1,1,0,1,0,0,0,1,1,0,0,1
9,8271,5177,0.988333,8.0,110,0.772411,20,1.119409,4919,8.865922,1,0,0,0,0,0,1,0,0,0,1,0,1525.538956,108.716667,1,0,0,0,0,0,0,7.666667,14,76.719048,0.797434,0,0,0,1,0,0,0,0,0,0,0,1,1,1


In [10]:
model2.fit(X_train, y_train)
submit1 = pd.DataFrame(model2.predict(X_test))
importances = model2.feature_importances_
indices = np.argsort(importances)[::-1]
feature_names=X_train.columns
print("Feature importances:")
for f, idx in enumerate(indices):
    print("{:2d}. feature '{:5}' ({:.4f})".format(f+1, feature_names[idx], importances[idx]))
  


Feature importances:
 1. feature 'watched_time_x' (0.9217)
 2. feature 'user_rating_mean' (0.0614)
 3. feature 'rating_mean' (0.0112)
 4. feature 'wt_x_user_mean' (0.0027)
 5. feature 'CM_S ' (0.0015)
 6. feature 'quantity_user_views' (0.0006)
 7. feature 'watched_time_y' (0.0003)
 8. feature 'wt_y_user_mean' (0.0003)
 9. feature 'quantity' (0.0001)
10. feature 'feature_3' (0.0001)
11. feature 'DEV0 ' (0.0001)
12. feature 'feature_2' (0.0001)
13. feature 'ts-f1' (0.0000)
14. feature 'user_uid' (0.0000)
15. feature 'duration' (0.0000)
16. feature 'element_uid' (0.0000)
17. feature 'CM_P ' (0.0000)
18. feature 'DEV5 ' (0.0000)
19. feature 'DEV1 ' (0.0000)
20. feature 'feature_4' (0.0000)
21. feature 'Q_duration=1' (0.0000)
22. feature 'feature_5_00' (0.0000)
23. feature 'Q_us_views=1' (0.0000)
24. feature 'feature_5_-1' (0.0000)
25. feature 'CM_R ' (0.0000)
26. feature 'feature_5_44' (0.0000)
27. feature 'DEV4 ' (0.0000)
28. feature 'feature_5_65' (0.0000)
29. feature 'feature_5_59' (0.0

In [ ]:
  
estimator=xgb.XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=700, n_jobs=-1  )    
estimator.fit(X_train, y_train)
submit2 = pd.DataFrame(estimator.predict(X_test))
importances = estimator.feature_importances_
indices = np.argsort(importances)[::-1]
feature_names=X_train.columns
print("Feature importances xgb:")
for f, idx in enumerate(indices):
    print("{:2d}. feature '{:5}' ({:.4f})".format(f+1, feature_names[idx], importances[idx]))

In [ ]:
3

In [11]:


X_test=X_test.drop([
'watched_time_x',
'watched',
'duration',
'feature_2',
'feature_3',
'user_rating_mean',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV2',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_duration=1',
'Q_duration=4',
'Q_wtx=1',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4',
'Q_us_views=1',
'Q_us_views=4'],1)
#submit[0]=submit[0].round()
#submit[0]=submit[0].astype(np.int16)
X_test=X_test.reset_index()
X_test=X_test.drop(['index'], 1)

OLD=pd.concat([X_test, submit1], 1)
#OLD.rating=OLD.rating.round()
OLD['rating']=OLD[0]
OLD=OLD.drop([0],1)
#ratings=ratings.drop(['ts'],1)
X_train = transactions_exp[~transactions_exp.rating.isnull()]
X_train=X_train.drop(['watched',
'watched_time_x',
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV2',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_us_views=1',
'Q_us_views=4',
'Q_duration=1',
'Q_duration=4',
'Q_wtx=1',
'user_rating_mean',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4'],1)
X_train=X_train.reset_index()
X_train=X_train.drop(['index'], 1)
OLD=OLD.append(X_train)
del OLD
OLD=pd.concat([X_test, submit1], 1)
#OLD.rating=OLD.rating.round()
OLD['rating']=OLD[0]
OLD=OLD.drop([0],1)

X_train=X_train.reset_index()
X_train=X_train.drop(['index'], 1)
OLD=OLD.append(X_train)
OLD.info()


In [13]:

OLD.to_csv("MovRatingRF.csv")

In [12]:
OLD.rating

0           8.122956
1           7.879739
2           8.122956
3           6.482792
4           8.436009
5           6.477361
6           6.483291
7           6.854381
8           6.022593
9           6.481486
10          6.492888
11          6.680198
12          7.630296
13          6.188875
14          5.827887
15          6.483611
16          5.322710
17          6.487126
18          6.515646
19          6.425221
20          6.559128
21          6.475863
22          6.250264
23          5.961268
24          6.201560
25         10.000000
26         10.000000
27         10.000000
28         10.000000
29         10.000000
             ...    
1875825     2.000000
1875826     4.000000
1875827     4.000000
1875828     2.000000
1875829     2.000000
1875830     2.000000
1875831     2.000000
1875832     2.000000
1875833     2.000000
1875834     2.000000
1875835     2.000000
1875836     2.000000
1875837     8.000000
1875838     2.000000
1875839     2.000000
1875840     2.000000
1875841     7

In [37]:
del OLD
OLD=pd.concat([X_test, submit1], 1)
#OLD.rating=OLD.rating.round()
OLD['rating']=OLD[0]
OLD=OLD.drop([0],1)

X_train=X_train.reset_index()
X_train=X_train.drop(['index'], 1)
OLD=OLD.append(X_train)
OLD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8447928 entries, 0 to 1875854
Data columns (total 3 columns):
element_uid    int32
user_uid       int32
rating         float64
dtypes: float64(1), int32(2)
memory usage: 193.4 MB


In [25]:
OLD.to_csv("MovRatingRF.csv")

In [36]:
X_train = transactions_exp[~transactions_exp.rating.isnull()]
X_train=X_train.drop(['watched',
'watched_time_x',
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV2',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_us_views=1',
'Q_us_views=4',
'Q_duration=1',
'Q_duration=4',
'Q_wtx=1',
'user_rating_mean',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4'],1)

In [26]:
del OLD2
OLD2=pd.concat([X_test, submit2], 1)
#OLD.rating=OLD.rating.round()
OLD2['rating']=OLD2[0]
OLD2=OLD2.drop([0],1)
OLD2=OLD2.append(X_train)
OLD2.info()
OLD2.to_csv("MovRatingXGB.csv")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8447928 entries, 0 to 1875854
Data columns (total 3 columns):
element_uid    int32
user_uid       int32
rating         float64
dtypes: float64(1), int32(2)
memory usage: 193.4 MB


In [29]:
OLD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8447928 entries, 0 to 1875854
Data columns (total 3 columns):
element_uid    int32
user_uid       int32
rating         float64
dtypes: float64(1), int32(2)
memory usage: 513.4 MB


In [21]:
OLD2.to_csv("MovRatingXGB.csv")

In [13]:
l=t_e[~t_e.rating.isnull()]
len(l)

349503

In [14]:
l[l.watched_time_x<0.4]

,element_uid,user_uid,watched_time_x,rating,duration,feature_2,feature_3,feature_4,quantity,rating_mean,CM_S,CM_P,CM_R,feature_5_65,feature_5_00,feature_5_68,feature_5_59,feature_5_-1,feature_5_44,smallf3,middlef3,bigf3,ts-f1,watched_time_y,DEV0,DEV1,DEV2,DEV3,DEV4,DEV5,DEV6,user_rating_mean,quantity_user_views,wt_y_user_mean,wt_x_user_mean,new_content,Q_el_views=1,Q_el_views=4,Q_us_views=1,Q_us_views=4,Q_duration=1,Q_duration=4,Q_wtx=1,Q_wtx=4,Q_f2=1,Q_f2=4,Q_f4=1,Q_f4=4
12,747,5177,0.358333,8.0,120,0.773630,44,1.140273,61272,8.930678,1,0,0,0,0,0,0,0,1,0,0,1,65.691293,43.000000,1,0,0,0,0,0,0,7.666667,14,76.719048,0.797434,0,0,1,1,0,0,1,0,0,0,0,0,0
113,3336,12353,0.051667,8.0,90,0.739609,45,1.141929,45646,8.328548,1,0,0,1,0,0,0,0,0,0,0,1,107.976843,4.650000,0,1,0,0,0,0,0,8.000000,5,6.426667,0.069017,0,0,1,1,0,1,0,1,0,0,0,0,0
116,813,12353,0.063750,8.0,80,0.677882,29,1.141929,21755,7.631387,1,0,0,0,0,0,1,0,0,0,1,0,87.969831,5.100000,0,1,0,0,0,0,0,8.000000,5,6.426667,0.069017,0,0,1,1,0,1,0,1,0,1,1,0,0
2000,9839,286819,0.386481,9.0,90,0.713218,12,1.135231,484,8.600000,1,0,0,0,0,0,1,0,0,1,0,0,838.403505,34.783333,0,0,0,0,0,1,0,8.571429,46,60.586232,0.626313,0,1,0,0,0,1,0,0,0,0,0,0,0
4601,5812,479460,0.020256,10.0,130,0.718939,35,1.136924,11669,8.005988,1,0,0,1,0,0,0,0,0,0,0,1,503.364410,2.633333,0,0,0,1,0,0,0,8.275862,156,48.016987,0.486160,0,0,0,0,1,0,1,1,0,0,0,0,0
4624,1277,479460,0.000000,7.0,120,0.705445,32,1.135231,7348,8.175610,1,0,0,0,0,0,1,0,0,0,1,0,719.115961,0.000000,0,0,0,0,1,0,0,8.275862,156,48.016987,0.486160,0,0,0,0,1,0,1,1,0,0,0,0,0
7068,9701,516947,0.278333,3.0,100,0.650878,25,1.141929,17543,7.253179,1,0,0,0,0,0,1,0,0,0,1,0,84.580381,27.833333,1,0,0,0,0,0,0,7.200000,32,80.384375,0.794317,0,0,0,0,0,0,0,0,0,1,1,0,0
9355,4441,122282,0.176852,10.0,180,0.720355,50,1.141929,19310,8.257718,0,1,0,1,0,0,0,0,0,0,0,1,0.451919,31.833333,1,0,0,0,0,0,0,8.333333,54,186.263580,1.586832,1,0,1,0,0,0,1,0,1,0,0,0,0
11508,72,197529,0.227500,10.0,100,0.761670,43,1.138604,36080,8.857282,1,0,0,0,0,0,0,0,1,0,0,1,286.323524,22.750000,1,0,0,0,0,0,0,10.000000,33,40.682828,0.419656,0,0,1,0,0,0,0,1,0,0,0,0,0
11678,2660,370328,0.111667,9.0,120,0.780857,26,1.133525,7651,8.571429,1,0,0,1,0,0,0,0,0,0,1,0,1050.958023,13.400000,1,0,0,0,0,0,0,7.781250,122,64.548770,0.655914,0,0,0,0,1,0,1,0,0,0,0,0,0


In [33]:
#a=pd.read_csv("MovRatingXGB1.csv")
b=pd.read_csv("MovRatingXGB5.csv")
c=pd.read_csv("MovRatingXGB4.csv")

In [30]:
a

,Unnamed: 0,element_uid,user_uid,rating
0,0,1911,5177,8.144646
1,1,1659,5177,7.720165
2,2,3783,5177,8.036150
3,3,4290,5177,6.448832
4,4,1789,5177,8.434494
5,5,3336,349202,4.841287
6,6,2639,349202,5.179202
7,7,7513,349202,4.998701
8,8,6397,349202,4.892508
9,9,2138,349202,5.363232


In [34]:
b

,Unnamed: 0,element_uid,user_uid,rating
0,0,1911,5177,8.070078
1,1,1659,5177,8.574954
2,2,3783,5177,7.904737
3,3,4290,5177,6.494210
4,4,1789,5177,8.396873
5,5,3336,349202,4.797360
6,6,2639,349202,5.216657
7,7,7513,349202,5.899892
8,8,6397,349202,4.878281
9,9,2138,349202,5.448943


In [35]:
c

,Unnamed: 0,element_uid,user_uid,rating
0,0,1911,5177,8.632125
1,1,1659,5177,9.064867
2,2,3783,5177,8.644176
3,3,4290,5177,6.643981
4,4,1789,5177,8.845860
5,5,3336,349202,8.256624
6,6,2639,349202,8.656743
7,7,7513,349202,9.347792
8,8,6397,349202,7.622055
9,9,2138,349202,8.594622
